In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

path="/content/drive/MyDrive/SentimentAnalysis/Sequence/ECTrain.csv"
df= pd.read_csv(path)
path1="/content/drive/MyDrive/SentimentAnalysis/Sequence/ECInd.csv"
df1= pd.read_csv(path1)

df.describe()

,Sequence,Target
count,13184,13184
unique,13184,2
top,OMTMSLEVFEKLEAKVQQAID,Positive
freq,1,6592


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
from keras.preprocessing import text, sequence
import re

In [4]:
print(df[ df['Target'] == 'Positive'].size)
print(df[ df['Target'] == 'Negative'].size)

print(df1[ df1['Target'] == 'Positive'].size)
print(df1[ df1['Target'] == 'Negative'].size)
for idx,row in df.iterrows():
    row[0] = str(row[0]).replace('rt','')
df.head()

13184
13184
722
2768


,Sequence,Target
0,OMTMSLEVFEKLEAKVQQAID,Positive
1,TLLQMEIEELKEKNNSLSQEV,Positive
2,EELERENNHLKEQQNGWQERL,Positive
3,REKIIASVAEKFICIADASKQ,Positive
4,EKFICIADASKQVDILGKFPL,Positive


In [5]:
max_fatures = 500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Sequence'].values)
X = tokenizer.texts_to_sequences(df['Sequence'].values)
X = pad_sequences(X)

In [6]:
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df1['Sequence'].values)
A = tokenizer.texts_to_sequences(df1['Sequence'].values)
A = pad_sequences(A)

In [7]:
Y = pd.get_dummies(df['Target']).values
B = pd.get_dummies(df1['Target']).values
X_train = X
Y_train = Y
X_test = A
Y_test = B

In [8]:
from tensorflow.keras.optimizers import SGD
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 128)            64000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 1, 128)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 319,194
Trainable params: 319,194
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
batch_size = 32
history=model.fit(X_train, Y_train, epochs =5, batch_size=batch_size, verbose = 2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Epoch 1/5
412/412 - 12s - loss: 0.6892 - accuracy: 0.5206 - 12s/epoch - 29ms/step
Epoch 2/5
412/412 - 4s - loss: 0.6782 - accuracy: 0.5204 - 4s/epoch - 10ms/step
Epoch 3/5
412/412 - 7s - loss: 0.6688 - accuracy: 0.5248 - 7s/epoch - 16ms/step
Epoch 4/5
412/412 - 5s - loss: 0.6671 - accuracy: 0.5342 - 5s/epoch - 13ms/step
Epoch 5/5
412/412 - 7s - loss: 0.6669 - accuracy: 0.5286 - 7s/epoch - 17ms/step
(13184, 1) (13184, 2)
(1745, 1) (1745, 2)


In [10]:
import numpy as np
Y_pred = np.argmax(model.predict(X_test,batch_size = batch_size), axis=-1)

In [11]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix \n",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix 
 [[1246  138]
 [   0  361]]
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      1384
           1       0.72      1.00      0.84       361

    accuracy                           0.92      1745
   macro avg       0.86      0.95      0.89      1745
weighted avg       0.94      0.92      0.93      1745



In [12]:
from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(df_test.true, df_test.pred)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(df_test.true, df_test.pred)

MCC:  0.807038594558181


array([[1246,  138],
       [   0,  361]])